# Batch Processing Example
This notebook demonstrates how to use the CloudProcessor for batch processing data with single or multiple workers.

## 1. Configuration
Set the project root path:

In [1]:
import os
from pathlib import Path
import json
from typing import Dict
import sys

def setup_project_path():
    """Add project root to Python path by searching for .git directory"""
    current_path = Path.cwd()
    
    # Search up the directory tree for .git folder or pyproject.toml
    root_indicators = ['.git', 'pyproject.toml']
    
    while current_path != current_path.parent:
        if any((current_path / indicator).exists() for indicator in root_indicators):
            sys.path.append(str(current_path))
            return current_path
        current_path = current_path.parent
    
    raise RuntimeError(
        "Could not find project root. "
        "Please run this notebook from within the project directory."
    )

# Setup path
project_root = setup_project_path()
print(f"Project root detected at: {project_root}")

Project root detected at: /home/steffen/sign-language-translator


Set your vertex ai configuration:

In [2]:
from models.vertex_ai import get_config

# Get vertex ai configuration for 'dev' environment
config = get_config("dev")  

Environment variables loaded from .env


Loading configuration from /home/steffen/sign-language-translator/models/vertex_ai/config/dev.yaml
Configuration loaded successfully!


## 2. Model Deployment

Define the model id and task:

In [3]:
hf_model_id = "openai/whisper-small"
hf_task = "automatic-speech-recognition"

Deploy the model:

In [ ]:
from models.vertex_ai import ModelDeployer
# Initialize deployer with your config
deployer = ModelDeployer(
    project_id=config.project_id,
    environment=config.environment
)

# Trigger the deployment and wait for completion
try:
    build_info = deployer.trigger_deployment(
        model_id=hf_model_id,
        hf_task=hf_task,
        endpoint_id="your-endpoint-id",  # Add your endpoint ID here
        branch="steffen74/issue18",
        hf_token=config.huggingface_token,
        trigger_name="test",
        wait_for_completion=True,  # This will make it wait
        timeout_minutes=45  # Adjust based on your expected build time
    )
    
    if build_info['status'] == 'SUCCESS':
        print("Deployment completed successfully!")
    else:
        print(f"Deployment failed with status: {build_info['status']}")
        
except Exception as e:
    print(f"Error during deployment: {e}")


To interrupt the model deployment:

In [ ]:
deployer.cancel_deployment("<build_id>") # Add the build id here

# The build id is printed at the beginning of the trigger deployment above.